# L4: Using the Model for Predictions

## Goal
Build a working Python Q&A assistant that answers questions in Stack Overflow style.

## Approach
Since fine-tuning was blocked in L3 (API unavailable), we're using:
- Base Model: Gemini 1.5 Flash
- Technique: Instruction-based prompting (zero-shot)
- Style: Stack Overflow developer answers

## What This Demonstrates
- Prompt engineering
- API integration
- Production-ready code structure
- Error handling



# L4: Using the Model for Predictions

## Attempt 1: Google Cloud Vertex AI ❌

**Issue:** No model access in project/region
**Result:** All models returned 404 errors
**Learning:** Regional limitations and access restrictions

Below is the original attempt (kept for documentation):

---

Setup (code)

# L4: Stack Overflow Python Q&A Assistant
# Using base Gemini model with Stack Overflow style prompting

from vertexai.generative_models import GenerativeModel
import vertexai
from typing import List, Dict
import datetime

print("🚀 Initializing Stack Overflow AI Assistant...")

Configuration (Code)

# Configuration
PROJECT_ID = "dotted-music-460617-k2"
REGION = "us-central1"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=REGION)

# Load base model
model = GenerativeModel("gemini-1.5-flash-002")

print(f"✅ Connected to project: {PROJECT_ID}")
print(f"✅ Region: {REGION}")
print(f"✅ Model loaded: gemini-1.5-flash-002")

Define Instruction Template (Code)

# Stack Overflow style instruction template
# This mimics what we would have trained the model on

INSTRUCTION_TEMPLATE = """You are a helpful Python programming expert who answers questions like a Stack Overflow developer.

When answering:
1. Be clear and concise
2. Provide working code examples
3. Explain WHY the solution works
4. Mention common pitfalls or alternatives
5. Use proper code formatting

Answer in Stack Overflow style - professional but friendly.

Question: {question}

Answer:"""

print("✅ Instruction template loaded")
print("\nTemplate preview:")
print("=" * 70)
print(INSTRUCTION_TEMPLATE.format(question="[Your question here]"))
print("=" * 70)

Create Assistant Function

def ask_python_question(question: str, temperature: float = 0.3) -> str:
    """
    Ask a Python question and get Stack Overflow style answer.
    
    Args:
        question (str): The Python question to ask
        temperature (float): Controls randomness (0.0-1.0)
                           Lower = more focused/deterministic
                           Higher = more creative/varied
    
    Returns:
        str: The answer from the model
    """
    try:
        # Format the prompt
        prompt = INSTRUCTION_TEMPLATE.format(question=question)
        
        # Generate response
        response = model.generate_content(
            prompt,
            generation_config={
                "temperature": temperature,
                "max_output_tokens": 1024,
            }
        )
        
        return response.text
        
    except Exception as e:
        return f"❌ Error: {str(e)}"

print("✅ Assistant function created")

Test with Single Question (code)

# Test 1: Simple question
print("=" * 70)
print("TEST 1: Simple CSV Reading Question")
print("=" * 70)

question = "How do I read a CSV file using pandas?"

print(f"\n Question: {question}\n")
print(" Answer:")
print("-" * 70)

answer = ask_python_question(question)
print(answer)

print("\n" + "=" * 70)

# Check what credentials the notebook is using
import google.auth

credentials, project = google.auth.default()

print("=" * 70)
print("NOTEBOOK AUTHENTICATION")
print("=" * 70)

print(f"\nProject from credentials: {project}")
print(f"Project in code: {PROJECT_ID}")
print(f"Match? {project == PROJECT_ID}")

print("\n" + "=" * 70)
print("CHECKING API ACCESS")
print("=" * 70)

# Check if we can access Vertex AI at all
from google.cloud import aiplatform

try:
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print("✅ Vertex AI connection successful")
except Exception as e:
    print(f"❌ Vertex AI error: {e}")

# Check available models
print("\n" + "=" * 70)
print("CHECKING MODEL ACCESS")
print("=" * 70)

try:
    # Try to list models
    from vertexai.preview.generative_models import GenerativeModel
    
    # Try different model names
    test_models = [
        "gemini-1.5-flash-002",
        "gemini-1.5-flash",
        "gemini-pro",
        "text-bison@001"
    ]
    
    for model_name in test_models:
        try:
            test_model = GenerativeModel(model_name)
            print(f"✅ {model_name} - Accessible")
        except Exception as e:
            print(f"❌ {model_name} - {str(e)[:80]}")
            
except Exception as e:
    print(f"Error checking models: {e}")

# Check authentication and model access
import google.auth
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel

# Get credentials
credentials, project = google.auth.default()

# Define project (in case Cell 3 wasn't run)
PROJECT_ID = "dotted-music-460617-k2"
REGION = "us-central1"

print("=" * 70)
print("NOTEBOOK AUTHENTICATION")
print("=" * 70)

print(f"\nProject from credentials: {project}")
print(f"Project in code: {PROJECT_ID}")
print(f"Match? {project == PROJECT_ID}")

print("\n" + "=" * 70)
print("CHECKING API ACCESS")
print("=" * 70)

# Check if we can access Vertex AI at all
try:
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print("✅ Vertex AI connection successful")
except Exception as e:
    print(f"❌ Vertex AI error: {e}")

# Check available models
print("\n" + "=" * 70)
print("CHECKING MODEL ACCESS")
print("=" * 70)

test_models = [
    "gemini-1.5-flash-002",
    "gemini-1.5-flash", 
    "gemini-1.5-pro",
    "gemini-pro",
    "text-bison@001",
    "text-bison@002"
]

for model_name in test_models:
    try:
        test_model = GenerativeModel(model_name)
        # Try a simple prediction
        response = test_model.generate_content("Say hello")
        print(f"✅ {model_name} - WORKS!")
    except Exception as e:
        error_msg = str(e)
        if "404" in error_msg or "not found" in error_msg:
            print(f"❌ {model_name} - Not available")
        elif "403" in error_msg or "permission" in error_msg.lower():
            print(f"⚠️  {model_name} - Permission denied")
        else:
            print(f"❌ {model_name} - {error_msg[:60]}")

print("\n" + "=" * 70)
print("DIAGNOSIS COMPLETE")
print("=" * 70)

---

## Attempt 2: OpenAI (Working Solution) ✅

After Google Cloud limitations, switching to OpenAI which provides:
- ✅ Immediate model access
- ✅ Reliable API
- ✅ Works with our fine-tuned model from L3

---

Setup OpenAI

In [2]:
# L4: Using OpenAI Fine-Tuned Model
import openai
import json
from typing import List, Dict
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ Please set OPENAI_API_KEY in .env file!")

openai.api_key = OPENAI_API_KEY

print("✅ OpenAI client initialized")
print(f"✅ API Key loaded: {OPENAI_API_KEY[:20]}...")  # Show first 20 chars only

✅ OpenAI client initialized
✅ API Key loaded: sk-proj-c4JHt8PTD-Go...


Load Fine-Tune Model

In [3]:
# Load your fine-tuned model name
# (This file was created in L3 after training completed)

try:
    with open("finetuned_model_name.txt", "r") as f:
        FINETUNED_MODEL = f.read().strip()
    
    print("✅ Fine-tuned model loaded!")
    print(f"📍 Model: {FINETUNED_MODEL}")
    
except FileNotFoundError:
    print("⚠️  Fine-tuned model file not found!")
    print("\nOptions:")
    print("1. Wait for L3 training to complete")
    print("2. Check finetuning_job_id.txt for status")
    print("3. Use base model for now")
    
    # Use base model for testing
    FINETUNED_MODEL = "gpt-4o-mini-2024-07-18"
    print(f"\n⚠️  Using base model for now: {FINETUNED_MODEL}")

⚠️  Fine-tuned model file not found!

Options:
1. Wait for L3 training to complete
2. Check finetuning_job_id.txt for status
3. Use base model for now

⚠️  Using base model for now: gpt-4o-mini-2024-07-18


CREATE ASSISTANT FUNCTION

In [7]:
def ask_python_question(question: str, temperature: float = 0.3) -> str:
    """
    Ask a Python question to your model.
    """
    try:
        response = openai.chat.completions.create(
            model=FINETUNED_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful Python programming expert who answers questions like a Stack Overflow developer."
                },
                {
                    "role": "user",
                    "content": question
                }
            ],
            temperature=temperature,
            max_tokens=1024
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f" Error: {str(e)}"

print(" Assistant function created")
print(" Ready to answer Python questions!")

 Assistant function created
 Ready to answer Python questions!


TEST IT

In [10]:
# Test 1: Simple question
print("=" * 70)
print("TEST 1: CSV Reading Question")
print("=" * 70)

question = "How do I read a CSV file using pandas?"

print(f"\n❓ Question: {question}\n")
print("🤖 Answer:")
print("-" * 70)

answer = ask_python_question(question)
print(answer)

print("\n" + "=" * 70)

TEST 1: CSV Reading Question

❓ Question: How do I read a CSV file using pandas?

🤖 Answer:
----------------------------------------------------------------------
To read a CSV file using the `pandas` library in Python, you can use the `pandas.read_csv()` function. Here's a step-by-step guide on how to do it:

1. **Install pandas**: If you haven't installed pandas yet, you can do so using pip:

   ```bash
   pip install pandas
   ```

2. **Import pandas**: In your Python script or interactive environment, import the pandas library.

   ```python
   import pandas as pd
   ```

3. **Read the CSV file**: Use the `pd.read_csv()` function to read the CSV file. You need to provide the file path as an argument.

   ```python
   df = pd.read_csv('path/to/your/file.csv')
   ```

   Replace `'path/to/your/file.csv'` with the actual path to your CSV file.

4. **View the DataFrame**: After reading the CSV file, you can view the contents of the DataFrame by simply printing it or using methods like 

In [1]:
# Check L3 fine-tuning status
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

try:
    # Load job ID
    with open("finetuning_job_id.txt", "r") as f:
        job_id = f.read().strip()
    
    # Get status
    job = openai.fine_tuning.jobs.retrieve(job_id)
    
    print("=" * 70)
    print("L3 FINE-TUNING STATUS")
    print("=" * 70)
    
    print(f"\n Job ID: {job.id}")
    print(f" Status: {job.status}")
    
    if job.status == "succeeded":
        print(f"\n TRAINING COMPLETE!")
        print(f" Your fine-tuned model: {job.fine_tuned_model}")
        
        # Save model name
        with open("finetuned_model_name.txt", "w") as f:
            f.write(job.fine_tuned_model)
        
        print("\n Model name saved!")
        print("\n Restart kernel and re-run from Cell 13!")
        
    elif job.status == "running":
        print("\n Still training...")
        print("   Check back in 10-15 minutes!")
        
    else:
        print(f"\n Status: {job.status}")
    
except FileNotFoundError:
    print("⚠️  finetuning_job_id.txt not found")
    print("   Did you complete L3 fine-tuning?")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70)

L3 FINE-TUNING STATUS

 Job ID: ftjob-F15zqv4WC6HSQWthkarxmgac
 Status: failed

 Status: failed

